# I. Аппроксимация методом наименьших квадратов (линейный случай).

Рассмотрим функцию $y = f(x)$ определенной набором значений $y_0, y_1, \cdots, y_n$ в точках $x_0, x_1, \cdots, x_n$.

In [ ]:
x = [-1, -0.7, -0.43, -0.14, -0.14, 0.43, 0.71, 1, 1.29, 1.57, 1.86, 2.14, 2.43, 2.71, 3]
y = [-2.25, -0.77, 0.21, 0.44, 0.64, 0.03, -0.22, -0.84, -1.2, -1.03, -0.37, 0.61, 2.67, 5.04, 8.90]

### I.I. Найдите многочлен наилучшего приближения.

$$
P_m(x) = a_0 + a_1 x + \cdots + a_m x^m
$$

используйте линейный метод наименьших квадратов. С его помощью

1. реализуйте алгоритм, создающий  "design matrix", используя $1, x, \cdots, x^m$ в качестве базисных функций.

2. постройте явно нормальную систему уравнений задачи наименьших квадратов при фиксированных значениях $m$.

3. решите систему уравнений, чтобы найти коэффициенты $P_m(x)$ для $m = 0, 1, 2, \dots$. Для решения задач линейной алгебры можете использовать либо библиотечные функции (`numpy.linalg.solve`), либо Вашу реализацию LU - разложение из 1 недели.

(20% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE

### I.II 

Чтобы найти оптимальное значение $m$, используйти следующий критерий: возьмем $m=0, 1, 2, \dots$, для каждого значения $m$ вычислим

$$
\sigma_m^2 = \frac{1}{n - m} \sum_{k=0}^n \left( P_m(x_k) - y_k \right)^2
$$

и возьмем такое значение $m$, при котором $\sigma_m$ стабилизируется или начинает расти.

(20% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

Постройте ваши полиномы $P_m(x)$ вместе с данными. Визуально сравните полиномы различных степеней. Согласуется ли визуальное сравнение с ожидаемым оптимальным значением $m$?

In [ ]:
# ... ENTER YOUR CODE HERE

### I.III. МНК с использовнием QR - разложениея.

Для оптимального значения $m$ из предыдущей части решите задачу МНК, используя безматричную реализацию QR - разложения. Для решения задач линейной алгебры можете использовать библиотечные функции (обратите внимание на `numpy.linalg.solve`, `numpy.linalg.qr` и т.д.) или код предыдущих недель.
Сравните результат с ответом, полученным при решении системы уравнений.

(20% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

# II. Интерполяция при помощи полиномов Лагранжа.

### II.1 

Рассмотрим функцию $f(x) = x^2 \cos{x}$. На интервале $x\in [\pi/2, \pi]$, интерполируйте функцию, используя полиномы Лагранжа степени $m$, где $m=1, 2, 3, 4, 5$. Используйте равномерную сетку. Постройте результат интерполяции вместе с $f(x)$.

(20% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

### II.2. 

Повторите предыдущее задание, используя узлы Чебышева. Сравните качество интерполяции на равномерной сетке и на узлах Чебышева для $m=3$.

(20% итоговой оценки).

In [ ]:
# ... ENTER YOUR CODE HERE ...